In [1]:
import pandas as pd
import numpy as np

In [2]:
import xml.etree.ElementTree
import warnings
warnings.filterwarnings('ignore')
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, SubElement, Comment
from xml.dom import minidom
import os

In [3]:
import matplotlib.pyplot as plt

In [4]:
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

init_notebook_mode(connected=True)
cf.go_offline()

## Processing Volume and Speed Estimate Data

In [7]:
df = pd.read_csv('VolumeEstimationData.csv')

In [8]:
df.head()

,EdgeID,begin,end,flow,speed,vType
0,-31294,0.0,720.0,15.818359,31.65,passenger
1,-31293,0.0,720.0,8.623181,33.40,passenger
2,-31291,0.0,720.0,7.948145,35.00,passenger
3,-31156,0.0,720.0,17.107676,21.50,passenger
4,-31042,0.0,720.0,9.417247,36.25,passenger


In [9]:
df['speed_ms'] = df['speed']/2.237

In [10]:
df.head()

,EdgeID,begin,end,flow,speed,vType,speed_ms
0,-31294,0.0,720.0,15.818359,31.65,passenger,14.148413
1,-31293,0.0,720.0,8.623181,33.40,passenger,14.930711
2,-31291,0.0,720.0,7.948145,35.00,passenger,15.645954
3,-31156,0.0,720.0,17.107676,21.50,passenger,9.611086
4,-31042,0.0,720.0,9.417247,36.25,passenger,16.204738


In [11]:
new_df = pd.DataFrame()

In [12]:
for e in df['EdgeID'].unique():
    tmp = pd.DataFrame()
    tmp = df[df['EdgeID']==e].copy()
    tmp['time'] = pd.to_datetime(tmp['begin'], unit='s')
    tmp = tmp.resample('1H', on='time').mean().reset_index()
    new_df = new_df.append(tmp.copy())

In [13]:
new_df.head()

,time,EdgeID,begin,end,flow,speed,speed_ms
0,1970-01-01 00:00:00,-31294,1440.0,2160.0,15.818359,31.65,14.148413
1,1970-01-01 01:00:00,-31294,5040.0,5760.0,14.891068,23.69,10.590076
2,1970-01-01 02:00:00,-31294,8640.0,9360.0,17.384167,24.23,10.831471
3,1970-01-01 03:00:00,-31294,12240.0,12960.0,18.969522,40.00,17.881091
4,1970-01-01 04:00:00,-31294,15840.0,16560.0,21.338293,39.15,17.501118


In [14]:
new_df.tail()

,time,EdgeID,begin,end,flow,speed,speed_ms
5,1970-01-01 05:00:00,31312,19440.0,20160.0,28.108545,35.000,15.645954
6,1970-01-01 06:00:00,31312,23040.0,23760.0,61.714816,22.835,10.207868
7,1970-01-01 07:00:00,31312,26640.0,27360.0,81.972751,24.270,10.849352
8,1970-01-01 08:00:00,31312,30240.0,30960.0,80.585190,25.620,11.452839
9,1970-01-01 09:00:00,31312,33480.0,34200.0,78.757770,23.320,10.424676


## Function to create additonal file with volume and speed calibrator

In [27]:
def build_additional_file(name,edges,data_df,hours=[0,24],interval=3600,freq=2):
    '''
    
    '''
    
    periods = np.arange(hours[0]*interval,hours[1]*interval,interval)
    
    additional = Element('additional')
    vtype = Element('vType')
    vtype.set('id', 'passenger')
    vtype.set('sigma', '0')
    vtype.set('vClass', 'passenger')
    
    additional.append(vtype)
  
    
    count = 0
    for edge in edges:
       
        ID_probe = "%s_%s_probe" %('tmp',count)
        routeProbe = Element('routeProbe')
        routeProbe.set('id', ID_probe)
        routeProbe.set('edge', str(edge))
        routeProbe.set('freq', str(freq))
        routeProbe.set('file',"route_probe_output.xml")
        ID = "%s_%s" %(edge,count)
        calibrator = Element('calibrator')
        calibrator.set('id', ID)
        calibrator.set('edge', str(edge))
        calibrator.set('pos', str(0))
        calibrator.set('output',"tmp.detector.xml")
        calibrator.set('freq',str(freq))
        calibrator.set('routeProbe',ID_probe)
        
        additional.append(routeProbe)
        additional.append(calibrator)
        ID_route = "%s_%s_fallback" %(edge,count)
        ET.SubElement(calibrator,"route",
                           id=ID_route, 
                           edges=str(edge))
        
        speed = data_df.loc[data_df['EdgeID'] == edge]['speed_ms']
        vols = data_df.loc[data_df['EdgeID'] == edge]['flow'].values
#         print('edge', edge, ' ', speed, '\n')
#         print(vols, '\n')
        
        for i,spd in enumerate(speed):
            b = periods[i]
            e = b + 3600
            ET.SubElement(calibrator,"flow",
                              begin=str(b), 
                              end=str(e), 
                              route=ID_route,
                              type='passenger',
                              vehsPerHour=str(vols[i]),
                              speed=str(spd))
  
        count+=1
#     Date = str(datetime.datetime.today()).split()[0]
    file_name = "cal.add1.%s.xml" %(name)
    folder = "./"
        
    with open(os.path.join(folder,file_name), 'wb') as f:
        f.write(minidom.parseString(ET.tostring(additional)).toprettyxml(encoding="utf-8"))
    print("your file named: ", file_name, " is located in: ",folder)
    return os.path.join(folder,file_name)

## Selecting Edges for Calibration and Creating Additional File

In [16]:
print(new_df[new_df['EdgeID']=='17321']['speed_ms'])

Series([], Name: speed_ms, dtype: float64)


In [17]:
edges1 = new_df['EdgeID'].unique()

In [18]:
print(edges1)

[-31294 -31293 -31291 -31156 -31042 -24909 -24859 -24852 -17323 -17322
 -17321 -17320 -17319 -17315 -17275 -17245 -16489 -16481 -16480 -16037
 -14392 -14390 -14324 -14323 -13907 -13901 -13825 -11375  -9904  -9685
  -9624  -9567  -9543  -2263  -2201  -1650  -1513  -1512  -1511  -1500
  -1499  -1492  -1491   -929   -928   -908   -907   -906   -643     79
    117   1477   1490   1491   1492   1495   1499   1500   1505   1508
   1512   1513   1517   1521   1527   1529   1650   2201   2202   2262
   2263   9567   9568   9624   9696   9794   9803   9839   9862  13693
  13694  13695  13904  13906  14323  14329  14331  15913  15914  15929
  16480  16481  16482  16489  17235  17275  17276  17315  17316  17319
  17321  17354  24794  24852  24859  24909  31156  31293  31294  31312]


In [20]:
# SUMO edge ids on Shallowford Rd
edges2 = [1505,-1505,1508,-1508,1512,-1512,1513,-1513,1517,-1517,-1527,-1527,1650,-1650,16951,-16951,16480,-16480,
        17276,-17276,17275,-17275,17235,-17235,17234,-17234,17315,-17315,17317,-17317,17316,-17316,17318,-17318,
         17319,-17319,17332,-17332, 17331, -17331, 17321,-17321,17320,-17320,17323,-17323,17322,-17322]

In [22]:
# Calibrate for only edges that appear in both the volume/speed data and shallowford Rd edges
edges = [value for value in edges1 if value in edges2]

In [23]:
print(edges)

[-17323, -17322, -17321, -17320, -17319, -17315, -17275, -16480, -1650, -1513, -1512, 1505, 1508, 1512, 1513, 1517, 1650, 16480, 17235, 17275, 17276, 17315, 17316, 17319, 17321]


In [24]:
# We calibrate from hour 0 to 10
hours = [0,10]

In [25]:
# Nprefix to the calibration additional file
name = 'test'

In [28]:
build_additional_file(name,edges,new_df,hours)

your file named:  cal.add1.test.xml  is located in:  ./


'./cal.add1.test.xml'

## Running Simulation with Calibrators and Visualizing Outputs

### Run micro simulation with SUMO calibrator

In [30]:
# Commented out to avoid excessive outputs from SUMO
# !cd sumo -n ShallowfordRd.net.xml -r ShallowfordRd.rou.xml -a cal.add1.test.xml,micro_get_out.xml

### Run meso simulation with calibrator 

In [31]:
# Run simulation with calibrator - Mesoscopic simulation (Commented out to avoid excessive outputs from SUMO)
# !cd input/ && sumo -n ShallowfordRd.net.xml -r ShallowfordRd.rou.xml -a cal.add1.test.xml,meso_get_out.xml --mesosim t

### Run micro simulation without calibration

In [32]:
# Running micro without calibration (Commented out to avoid excessive outputs from SUMO)
# !cd input/ && sumo -n ShallowfordRd.net.xml -r ShallowfordRd.rou.xml -a micro_no_cal.xml

### Run meso simulation without calibration

In [33]:
# Run simulation with no calibrator - Mesoscopic simulation (Commented out to avoid excessive outputs from SUMO)
# !cd input/ && sumo -n ShallowfordRd.net.xml -r ShallowfordRd.rou.xml -a meso_no_cal.xml --mesosim t

In [27]:
# Convert micro output xml into csv -- with calibration
!cd output/ && python $SUMO_HOME/tools/xml/xml2csv.py micro_output.xml
micro_out = pd.read_csv('output/micro_output.csv',sep=';')

In [28]:
# Convert meso output xml into csv  -- with calibration
!cd output/ && python $SUMO_HOME/tools/xml/xml2csv.py meso_output.xml
meso_out = pd.read_csv('output/meso_output.csv',sep=';')

In [29]:
# Convert micro output xml into csv
!cd output/ && python $SUMO_HOME/tools/xml/xml2csv.py micro_no_out.xml
micro_out_b = pd.read_csv('output/micro_no_out.csv',sep=';')

In [30]:
# Convert meso output xml into csv
!cd output/ && python $SUMO_HOME/tools/xml/xml2csv.py meso_no_out.xml
meso_out_b = pd.read_csv('output/meso_no_out.csv',sep=';')

In [31]:
# Calculate Average traffic volume (#/h) = speed * 3.6 * density

In [32]:
micro_out['volume'] = micro_out['edge_speed']*3.6*micro_out['edge_density']
meso_out['volume'] = meso_out['edge_speed']*3.6*meso_out['edge_density']

In [33]:
micro_out_b['volume'] = micro_out_b['edge_speed']*3.6*micro_out_b['edge_density']
meso_out_b['volume'] = meso_out_b['edge_speed']*3.6*meso_out_b['edge_density']

In [34]:
micro_out.head()

,interval_begin,interval_end,interval_id,edge_arrived,edge_density,edge_departed,edge_entered,edge_id,edge_laneChangedFrom,edge_laneChangedTo,edge_laneDensity,edge_left,edge_occupancy,edge_overlapTraveltime,edge_sampledSeconds,edge_speed,edge_traveltime,edge_waitingTime,volume
0,0.0,3600.0,congestion,3,0.77,20,4,-11375,0,0,0.77,18,0.38,47.22,1005.29,7.81,46.37,0.0,21.64932
1,0.0,3600.0,congestion,0,0.75,13,14,-13693,0,0,0.75,27,0.35,5.23,134.89,10.55,4.89,0.0,28.48500
2,0.0,3600.0,congestion,0,0.47,14,0,-13694,0,0,0.47,14,0.23,7.97,102.79,8.18,7.65,0.0,13.84056
3,0.0,3600.0,congestion,2,3.27,19,128,-13695,0,0,3.27,145,1.50,4.09,594.57,13.57,3.78,0.0,159.74604
4,0.0,3600.0,congestion,4,2.09,16,39,-13825,33,33,1.04,51,0.50,21.88,1191.61,7.48,20.55,1.0,56.27952


In [35]:
meso_out.head()

,interval_begin,interval_end,interval_id,edge_arrived,edge_density,edge_departed,edge_entered,edge_id,edge_laneChangedFrom,edge_laneChangedTo,edge_laneDensity,edge_left,edge_occupancy,edge_overlapTraveltime,edge_sampledSeconds,edge_speed,edge_traveltime,edge_waitingTime,volume
0,0.0,3600.0,congestion,3,0.68,20,4,-11375,0,0,0.68,18,0.34,41.74,894.16,8.84,41.17,0.0,21.64032
1,0.0,3600.0,congestion,0,0.43,13,14,-13693,0,0,0.43,27,0.22,3.16,77.64,17.44,2.88,0.0,26.99712
2,0.0,3600.0,congestion,0,0.23,14,0,-13694,0,0,0.23,14,0.11,3.82,49.35,17.07,3.52,0.0,14.13396
3,0.0,3600.0,congestion,2,2.39,19,129,-13695,0,0,2.39,145,1.19,3.24,434.18,17.14,2.95,0.0,147.47256
4,0.0,3600.0,congestion,4,1.85,16,44,-13825,0,0,0.93,56,0.46,18.19,1058.01,9.00,17.63,0.0,59.94000


In [36]:
# Calculating MAPE for calibrated simulations
micro_err = {}
micro_err_v = {}
meso_err = {}
meso_err_v = {}
for edge in edges:
    # Speed MAPE
    micro_spd = micro_out[micro_out['edge_id']==str(edge)]['edge_speed'].values[1:9]
    meso_spd = meso_out[meso_out['edge_id']==str(edge)]['edge_speed'].values[1:9]
    tt_spd = new_df.loc[new_df['EdgeID'] == edge]['speed_ms'].values[1:9]
    micro_error = np.mean(np.abs(micro_spd - tt_spd)/tt_spd) *100
    meso_error = np.mean(np.abs(meso_spd - tt_spd)/tt_spd) *100
    micro_err[str(edge)] = micro_error
    meso_err[str(edge)] = meso_error
    
    # Volume MAPE
    micro_vol = micro_out[micro_out['edge_id']==str(edge)]['volume'].values[1:9]
    meso_vol = meso_out[meso_out['edge_id']==str(edge)]['volume'].values[1:9]
    tt_vol = new_df.loc[new_df['EdgeID'] == edge]['flow'].values[1:9]
    micro_error = np.mean(np.abs(micro_vol - tt_vol)/tt_vol) *100
    meso_error = np.mean(np.abs(meso_vol - tt_vol)/tt_vol) *100
    micro_err_v[str(edge)] = micro_error
    meso_err_v[str(edge)] = meso_error
#     print("edge: ", edge, " error: ", error, "\n")

In [37]:
# Calculating MAPE for basiline simulations
micro_err_b = {}
micro_err_v_b = {}
meso_err_b = {}
meso_err_v_b = {}
for edge in edges:
    # Speed MAPE
    micro_spd = micro_out_b[micro_out_b['edge_id']==str(edge)]['edge_speed'].values[1:9]
    meso_spd = meso_out_b[meso_out_b['edge_id']==str(edge)]['edge_speed'].values[1:9]
    tt_spd = new_df.loc[new_df['EdgeID'] == edge]['speed_ms'].values[1:9]
    micro_error = np.mean(np.abs(micro_spd - tt_spd)/tt_spd) *100
    meso_error = np.mean(np.abs(meso_spd - tt_spd)/tt_spd) *100
    micro_err_b[str(edge)] = micro_error
    meso_err_b[str(edge)] = meso_error
    
    # Volume MAPE
    micro_vol = micro_out_b[micro_out_b['edge_id']==str(edge)]['volume'].values[1:9]
    meso_vol = meso_out_b[meso_out_b['edge_id']==str(edge)]['volume'].values[1:9]
    tt_vol = new_df.loc[new_df['EdgeID'] == edge]['flow'].values[1:9]
    micro_error = np.mean(np.abs(micro_vol - tt_vol)/tt_vol) *100
    meso_error = np.mean(np.abs(meso_vol - tt_vol)/tt_vol) *100
    micro_err_v_b[str(edge)] = micro_error
    meso_err_v_b[str(edge)] = meso_error

In [38]:
df_box = pd.DataFrame()

In [39]:
print(micro_err.values())

dict_values([22.048413256829168, 80.41754970098796, 15.759862631505891, 82.65355553447289, 22.27019889930339, 25.458939434818404, 67.11797862216811, 92.80873684192234, 62.194678655685344, 17.152828624502114, 67.45467953727446, 18.065774854780013, 25.35278361573983, 24.077892358946045, 87.41261712888861, 14.424105245522389, 83.84783850465982, 19.28648119875645, 72.93593202234301, 27.89347037254496, 59.49162749541729, 90.03880144295552, 35.274293943105626, 39.43708290427234, 16.757226401523738])


In [40]:
df_box['Base Micro Speed'] = list(micro_err_b.values())
df_box['Calibrated Micro Speed'] = list(micro_err.values())
df_box['Base Micro Volume'] = list(micro_err_v_b.values())
df_box['Calibrated Micro Volume'] = list(micro_err_v.values())

In [41]:
df_box['Base Meso Speed'] = list(meso_err_b.values())
df_box['Calibrated Meso Speed'] = list(meso_err.values())
df_box['Base Meso Volume'] = list(meso_err_v_b.values())
df_box['Calibrated Meso Volume'] = list(meso_err_v.values())

In [42]:
df_box.iplot(kind='box')